## EECS 453/551
# Video background subtraction using SVD

In this exercise we will discover how the SVD can magically estimate the background in a video even when there is a lot of noise and missing data.
___

## Before we begin

2. Run the code cell below by clicking it and either

    * choosing Cell -> Run in the toolbar
    * Pressing Ctrl+Enter

This will load the Python tools we need for today.

In [ ]:
# %cd background_demo
%pylab inline
from scipy.io import loadmat
from scipy.sparse.linalg import svds
from ipywidgets import interact

You may wish to take a few minutes to familiarize yourself with the notebook interface. Choose Help -> User Interface Tour for a brief introduction.

In Julia, you can type `?` followed by any object (variable, function, module) to get its description.

Now run the next cell. You should see an image with a slider above it. Drag the slider to look through some video footage.

In [ ]:
# load lobby and xylophone video footage:
lobby = loadmat("lobby.mat")["MovMat"]

xylophone = loadmat("xylophone.mat")["MovMat"]
xylophone = xylophone.astype(float)
xylophone *= 1/xylophone.max()

# display footage:
footage = lobby

m,n,numFrames = footage.shape

fig = figure(figsize = [10,10])
ax = axes(xticks=[],yticks=[])
set_cmap('gray')

def on_change(val):
    imshow(footage[:,:,val])
    xticks([])
    yticks([])
    figsize(10,10)
    show()

interact(on_change, val=(0,numFrames-1))

## Objective

The objective of this exercise is to take as an input the video you have just seen and return as an output the background (i.e. without the people moving for the lobby video).

### Task 1

Find in the function below the portion where the variable `backgroundVec` is computed. Note how the SVD is used to compute it. Examine how the `residualVec` variable is computed and note that

    footageVec = backgroundVec + residualVec

Hence if we successfully are able to estimate the background, `residualVec` should contain:

1. People moving around
2. Nothing (empty)

**Do you expect 1. or 2. to be true? Why?**

Now **run the cell** to add `process()` to your workspace.

In [ ]:
def process(footage):
    """
    process(footage) -> background,residual
    Use SVD to separate input footage into background (static objects)
    and residual (fleeting objects).
    """

    (m,n,numFrames) = footage.shape
    
    # Each movie frame is a 128x160 array.
    # Reshape each frame into a single column vector
    # to ease manipulation:
    footageVec = footage.reshape((m*n,numFrames))
    
    ###################################
    # Process using SVD
    r = 1 # rank of "background" is 1

    # U,S,V will contain first r components of SVD:
    
    [U,S,V] = svds(footageVec,r) # V already transposed!
    
    # Take first 'r' components as background
    backgroundVec = U.dot(diag(S)).dot(V) 

    # calculate residual
    residualVec = footageVec - backgroundVec
    ###################################
    
    # reshape background and residual into footage dimensions
    background = backgroundVec.reshape((m,n,numFrames))
    residual = residualVec.reshape((m,n,numFrames))
    return background,residual

### Task 2

Run the following code cell. You should see a slider above a plot with two subplots. On the left is the original footage; on the right is the background as estimated by SVD.

**Is it working?**

In [ ]:
footage = lobby

# perform SVD-based background subtraction:
background,residual = process(footage)

### display results
m,n,numFrames = footage.shape
ws = ones((m,5)) # whitespace

fig = figure(figsize = [20,10])
ax = axes(xticks=[],yticks=[])
set_cmap('gray')

def on_change(val):
    imshow(hstack((footage[:,:,val],ws,background[:,:,val])))
    xticks([])
    yticks([])
    figsize(20,10)
    show()

interact(on_change, val=(0,numFrames-1))

### Task 3: adding noise

The movie you just processed using SVD was relatively noise-free. Let us start adding noise and removing entries and see how well SVD does.

Study the function `addnoise()` in the following cell. **What is the code doing mathematically?** Now **run the cell** to add the function to your workspace.

Enter a value of 0.1 for $ \sigma $ in the next cell down, then run it.

**Do you see the movie get noisier? Is the SVD able to extract the background?**

* Now repeat for $ \sigma = 0.5 $, $ \sigma = 1 $ and $ \sigma = 2 $. **How does the SVD do?**

In [ ]:
def addnoise(footage,sigma):
    """
    Add noise to footage.

    σ is noise standard deviation.

    (type symbol with "\sigma<TAB>")
    """
    rand
    # Interpret this mathematically!
    footage = footage + sigma*rand(*footage.shape)
    return footage

In [ ]:
footage = lobby

############################
sigma =  # Enter your value here

# Add noise:
footage = addnoise(footage,sigma)
############################

# perform background subtraction on noisy footage:
background,residual = process(footage)

### display results
m,n,numFrames = footage.shape
ws = ones((m,5)) # whitespace

fig = figure(figsize = [20,10])
ax = axes(xticks=[],yticks=[])
set_cmap('gray')

def on_change(val):
    imshow(hstack((footage[:,:,val],ws,background[:,:,val])))
    xticks([])
    yticks([])
    figsize(20,10)
    show()

interact(on_change, val=(0,numFrames-1))

### Task 4: removing entries

Now we will corrupt the data by randomly removing a portion of the entries.

Study `corrupt()` in the cell below. **What is it doing to the input footage?** Now **run the cell** to add `corrupt()` to your workspace.

Enter a value of 0.9 for `p` in the next cell down, then run it.

**Does the movie have missing entries? Is the SVD successfully separating the background?**

Now repeat for `p = 0.75`, `p = 0.5`, `p = 0.25`, `p = 0.1`. **Do you see the movie have more blank entries? Is the SVD still working?**

In [ ]:
def corrupt(footage,p):
    """
    Corrupt footage by removing entries.

    Each entry is removed with probability `p`.
    """
    if p < 0.0 or p > 1.0:
        warnings.warn("p must be in [0,1]")
    
    # Interpret this mathematically!
    # Type help(rand) if you aren't sure what rand() does.
    mask = rand(*footage.shape) < p

    # Mask some entries
    footage = footage*mask
    return footage

In [ ]:
footage = lobby

# Add noise:
sigma = 0.1
footage = addnoise(footage,sigma)

############################
p =  # Enter your value here

# Delete entries
footage = corrupt(footage,p)
############################

# Process footage using SVD
background,res = process(footage)

### display results
m,n,numFrames = footage.shape
ws = ones((m,5)) # whitespace

fig = figure(figsize = [20,10])
ax = axes(xticks=[],yticks=[])
set_cmap('gray')

def on_change(val):
    imshow(hstack((footage[:,:,val],ws,background[:,:,val])))
    xticks([])
    yticks([])
    figsize(20,10)
    show()

interact(on_change, val=(0,numFrames-1))

The corrupted footage is on the left; the background (first component of SVD) is on the right.

## Task 5

What if we considered the first two SVD components to be the background rather than just one? What happens as we change $\sigma$ and p?

The following method returns both rank-1 and rank-2 backgrounds and allows us to quickly change $\sigma$ and p. **Run the cell** to add this method to your workspace.

In [ ]:
def process2(footage,sigma,p):
    (m,n,numFrames) = footage.shape
    footage = addnoise(footage,sigma) # Add noise
    footage = corrupt(footage,p) # Delete entries
     # Reshape each frame into a column
    footageVec = footage.reshape(m*n,numFrames)
    
    [U,S,V] = svds(footageVec,2) # take SVD
    
    # take first component as background
    backgroundVec = S[-1]*outer(U[:,-1],V[-1,:])
    
    # take first 2 components as background2
    backgroundVec2 = U.dot(diag(S)).dot(V)
    
    # calculate residual
    residualVec = footageVec - backgroundVec
    
    # calculate residual2
    residualVec2 = footageVec - backgroundVec2  
    
    # reshape footage dimensions
    background  = backgroundVec.reshape(footage.shape)
    background2 = backgroundVec2.reshape(footage.shape)
    
    return footage,background,background2

Now we can illustrate the difference between rank-1 and rank-2 background separation. **Run the following cell** to see footage on the left, rank-1 background in the middle, and rank-2 background on the right.

In [ ]:
###########################
# Enter your values here
sigma = 
p = 
footage = lobby # xylophone
###########################

# perform background subtraction
fg,bg,bg2 = process2(footage,sigma,p)

### display results
m,n,numFrames = footage.shape
ws = ones((m,5)) # whitespace

fig = figure(figsize = [20,10])
ax = axes(xticks=[],yticks=[])
set_cmap('gray')

def on_change(val):
    imshow(hstack((fg[:,:,val],ws,
                   bg[:,:,val],ws,bg2[:,:,val])))
    xticks([])
    yticks([])
    figsize(20,10)
    show()

interact(on_change, val=(0,numFrames-1))

### Task 6

In the cell above, change 
```python
footage = lobby
```
to 
```python
footage = xylophone
```
and run the cell again. **Does SVD-based processing still work?**